<a href="https://colab.research.google.com/github/malhardamle/nba_parlays_vision_model/blob/main/nba_parlay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install torch transformers huggingface_hub bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [11]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "llava-hf/llava-1.5-7b-hf"

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(model_id)


🔄 Initializing EasyOCR...
🔄 Loading Mistral model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Step 1: Install required packages
!pip install -q torch transformers accelerate bitsandbytes huggingface_hub

# Step 2: Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import notebook_login

# Step 3: Authenticate with Hugging Face
notebook_login()

# Step 4: Load model with GPU optimization
model_name = "mistralai/Mistral-7B-v0.1"

print("🚀 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("⚡ Loading model (this may take a few minutes)...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,  # Quantization for reduced memory usage
    low_cpu_mem_usage=True
)

# Step 5: Create generation function
def generate_response(prompt, max_length=150):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        temperature=0.7,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 6: Test the model
test_prompt = "Explain parlays in sports betting in simple terms."
print(f"\n📝 Prompt: {test_prompt}")

response = generate_response(test_prompt)
print("\n💡 Mistral's Response:")
print("-" * 80)
print(response)
print("-" * 80)

# Step 7: Try your own prompts
while True:
    user_input = input("\nAsk me about sports betting (or type 'quit' to exit): ")
    if user_input.lower() == 'quit':
        break
    print("\n🤖 Generating response...")
    print("-" * 80)
    print(generate_response(user_input))
    print("-" * 80)

In [5]:
# Parlay Analyzer with Mistral-7B and EasyOCR
# Complete Google Colab Script with Side-by-Side Results

# Step 1: Install required packages
!pip install -q torch transformers accelerate bitsandbytes easyocr gradio Pillow huggingface_hub

# Step 2: Imports
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import torch
import easyocr
import gradio as gr
from PIL import Image
import numpy as np
import re
import warnings
from google.colab import userdata

# Suppress warnings
warnings.filterwarnings('ignore')

# Step 3: Secure Hugging Face Login
print("🔑 Authenticating with Hugging Face...")
try:
    HF_TOKEN = userdata.get('HF_TOKEN')  # Get from Colab secrets
    login(token=HF_TOKEN)
    print("✅ Hugging Face login successful!")
except Exception as e:
    print(f"❌ Authentication failed: {str(e)}")
    print("\nℹ️ Setup instructions:")
    print("1. Get your Hugging Face token from: https://huggingface.co/settings/tokens")
    print("2. Click the 'key' icon in Colab's left sidebar")
    print("3. Add a new secret named 'HF_TOKEN' with your token")
    print("4. Accept model license at: https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1")
    raise

# Step 4: Initialize EasyOCR
print("🔄 Initializing EasyOCR reader...")
try:
    reader = easyocr.Reader(['en'])
    print("✅ EasyOCR ready!")
except Exception as e:
    print(f"❌ EasyOCR initialization failed: {str(e)}")
    raise

# Step 5: Load Mistral Model
print("🔄 Loading Mistral-7B model (this may take a few minutes)...")
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

try:
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)

    # Configure 4-bit quantization
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quant_config,
        device_map="auto",
        torch_dtype=torch.float16,
        token=HF_TOKEN
    )
    print("✅ Mistral-7B model loaded successfully!")
except Exception as e:
    print(f"❌ Model loading failed: {str(e)}")
    print("\nℹ️ Troubleshooting tips:")
    print("- Ensure you accepted the model license agreement")
    print("- Check your Hugging Face token has correct permissions")
    print("- Try using a Colab Pro with high-RAM runtime")
    raise

# Step 6: Analysis Function
def generate_analysis(player_bets):
    try:
        prompt = f"""Analyze these basketball player props and provide:
1. Risk assessment (High/Medium/Low) with reasoning
2. Suggested adjustments if High risk
3. Final recommendation

Player Props:
{player_bets}

Respond in this exact format:
[Player Name] - [Risk Level]: [Brief Analysis]
[Suggested Adjustment] (if High Risk)

Final Verdict: [Keep/Drop] because [1-2 sentence explanation]"""

        # Tokenize and generate
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()
    except Exception as e:
        return f"⚠️ Analysis error: {str(e)}"

# Step 7: Image Processing Pipeline
def process_image(image):
    try:
        # Convert and validate image
        if not isinstance(image, (np.ndarray, Image.Image)):
            raise ValueError("Invalid image format")

        if isinstance(image, Image.Image):
            image = np.array(image)

        # OCR processing
        results = reader.readtext(image, detail=0, paragraph=False)
        raw_text = "\n".join([res for res in results if len(res.strip()) > 2])

        if not raw_text:
            return "No text detected", "Please upload a clearer image"

        # Extract player props
        player_bets = re.findall(
            r'([A-Z][a-zA-Z\s\-\.\']+?\s\d{1,2}\s*[+\-±]\s*\d*\.?\d*\s*(Points|Rebounds|Assists|PRA|PR|PA|RA))',
            raw_text,
            re.IGNORECASE
        )
        player_bets = [bet[0].strip() for bet in player_bets if bet[0].strip()]

        if not player_bets:
            return "Extracted Text", f"No player props found. Full text:\n{raw_text[:500]}..."

        # Run analysis (limit to 6 props for performance)
        analysis = generate_analysis("\n".join(player_bets[:6]))

        # Format output
        if "Final Verdict:" in analysis:
            parts = analysis.split("Final Verdict:")
            analysis_part = parts[0].strip()
            verdict_part = f"\n\n🔮 Final Verdict: {parts[1].strip()}"
        else:
            analysis_part = analysis
            verdict_part = "\n\n⚠️ No clear verdict generated"

        return "🎯 Detected Props:\n" + "\n".join(player_bets[:6]), f"📊 Analysis:\n{analysis_part}{verdict_part}"

    except Exception as e:
        return "❌ Error", f"Processing failed: {str(e)}"

# Step 8: Gradio UI with Side-by-Side Results
with gr.Blocks(title="🏀 Parlay Prop Analyzer", theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🏀 Parlay Prop Analyzer
    *Upload an image of your bet slip for AI-powered risk analysis*
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📷 Upload Your Bet Slip")
            image_input = gr.Image(type="pil", label="Drag image here")
            analyze_btn = gr.Button("Analyze Risk", variant="primary")

        with gr.Column(scale=2):
            gr.Markdown("### 📝 Analysis Results")
            with gr.Row():
                # Left box for detected props
                extracted_output = gr.Textbox(
                    label="Detected Player Props",
                    lines=8,
                    interactive=False,
                    container=True
                )
                # Right box for analysis
                analysis_output = gr.Textbox(
                    label="Risk Analysis",
                    lines=8,
                    interactive=False,
                    container=True
                )

    analyze_btn.click(
        fn=process_image,
        inputs=image_input,
        outputs=[extracted_output, analysis_output],
        api_name="analyze"
    )

# Step 9: Launch App
print("\n🚀 Launching Parlay Analyzer...")
try:
    app.launch(share=True, debug=False)
    print("\n✅ App is running! Share the public URL with others")
except Exception as e:
    print(f"\n❌ Failed to launch app: {str(e)}")
    print("ℹ️ Try restarting the runtime or using a different port")

🔑 Authenticating with Hugging Face...
✅ Hugging Face login successful!
🔄 Initializing EasyOCR reader...
✅ EasyOCR ready!
🔄 Loading Mistral-7B model (this may take a few minutes)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Mistral-7B model loaded successfully!

🚀 Launching Parlay Analyzer...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5ee57bf530e777c7b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



✅ App is running! Share the public URL with others
